# Solution Optimization Evaluaton Raw TextGrad

In [1]:
import pandas as pd
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifier
from textgrad.loss import TextLoss

## Load Datasets

In [2]:
initial_solution = pd.read_csv("csv/initial_solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,1,\n Two quantum states with energies E1 and ...,<Solution>\nThe energy-time uncertainty princi...,B,GPQA-Diamond,-
1,2,\n trans-cinnamaldehyde was treated with me...,<Solution>\n1. **Reaction 1:** trans-cinnamald...,A,GPQA-Diamond,-
2,8,\n You perform a high-throughput experiment...,<Solution>\nHere's how we can analyze the resu...,B,GPQA-Diamond,-
3,3,\n A spin-half particle is in a linear supe...,<Solution>\nLet the given state be $|\psi\rang...,D,GPQA-Diamond,-
4,6,\n A quantum mechanical particle of mass m ...,"<Solution>\nThe potential is given by:\nV(r, θ...",D,GPQA-Diamond,-
...,...,...,...,...,...,...
407,404,"\n Under certain conditions, a beam of elec...",<Solution>\nThe de Broglie wavelength of the e...,C,MMLU-CP,college_physics
408,384,\n Two spaceships approach Earth with equal...,<Solution>\nLet $v$ be the speed of each space...,C,MMLU-CP,college_physics
409,411,\n Positronium is an atom formed by an elec...,<Solution>\nThe energy levels of positronium a...,C,MMLU-CP,college_physics
410,396,\n A photon strikes an electron of mass m t...,<Solution>\nLet $E_\gamma$ be the energy of th...,C,MMLU-CP,college_physics


In [3]:
# Test size only 50 rows each datasets (Total 150 rows)

df_gpqa = initial_solution[initial_solution['source'] == 'GPQA-Diamond'].head(50)
df_mmlu_ml = initial_solution[initial_solution['source'] == 'MMLU-ML'].head(50)
df_mmlu_cp = initial_solution[initial_solution['source'] == 'MMLU-CP'].head(50)
df_test = pd.concat([df_gpqa, df_mmlu_ml, df_mmlu_cp], ignore_index=True)

df_test

,id,formatted_question,raw_solution,correct_answer,source,subject
0,1,\n Two quantum states with energies E1 and ...,<Solution>\nThe energy-time uncertainty princi...,B,GPQA-Diamond,-
1,2,\n trans-cinnamaldehyde was treated with me...,<Solution>\n1. **Reaction 1:** trans-cinnamald...,A,GPQA-Diamond,-
2,8,\n You perform a high-throughput experiment...,<Solution>\nHere's how we can analyze the resu...,B,GPQA-Diamond,-
3,3,\n A spin-half particle is in a linear supe...,<Solution>\nLet the given state be $|\psi\rang...,D,GPQA-Diamond,-
4,6,\n A quantum mechanical particle of mass m ...,"<Solution>\nThe potential is given by:\nV(r, θ...",D,GPQA-Diamond,-
...,...,...,...,...,...,...
145,361,\n A grating spectrometer can just barely r...,<Solution>\nThe resolving power R of a grating...,C,MMLU-CP,college_physics
146,339,\n A proton moves in the +z-direction after...,<Solution>\nHere's how to solve this problem:\...,B,MMLU-CP,college_physics
147,364,"\n A net force F_A acts on object A, and a ...",<Solution>\nLet m_A be the mass of object A an...,B,MMLU-CP,college_physics
148,348,"\n The negative muon, mu^-, has properties ...",<Solution>\nA muon (μ) is a fundamental subato...,B,MMLU-CP,college_physics


## Experiment

In [4]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro")

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def evaluate_with_raw_textgrad(row_data):
    result = {
        "id": row_data["id"],
        "raw_solution": row_data["raw_solution"],
        "correct_answer": row_data["correct_answer"],
        "source": row_data["source"],
        "subject": row_data["subject"]
    }
    
    solution = Variable(row_data["raw_solution"],
                    requires_grad=True,
                    role_description="Solution to the math question")
    loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                    Do not attempt to solve it yourself, do not give a solution, 
                                    only identify errors. Be super concise.""",
                                    requires_grad=False,
                                    role_description="system prompt")
    optimizer = TextualGradientDescent([solution])
    loss = TextLoss(loss_system_prompt, engine=engine)
    
    # Iterate 5 times
    for i in range(1, 6):
        optimizer.zero_grad()  # Clean gradients
        loss_result = loss(solution)
        
        loss_result.backward()
        optimizer.step()
        result[f"solution_{i}"] = solution.value

    return result

## Running Evaluation

### Raw TextGrad

In [6]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

results = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=8) as executor:
    # Submit all tasks
    futures = [
        executor.submit(evaluate_with_raw_textgrad, row.to_dict()) 
        for _, row in df_test.iterrows()
    ]
    
    # Use tqdm for progress tracking
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
        result = future.result()
        if result is not None:
            results.append(result)

raw_textgrad = pd.DataFrame(results)

print(f"Completed in {time.time() - start_time:.1f} seconds")
raw_textgrad.to_csv('csv/raw_textgrad.csv', index=False)

Processing:   1%|▏         | 2/150 [00:00<00:08, 18.44it/s]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nThe potential is given by:\nV(r, θ) = (1/2)kr^2 + (3/2)kr^2 cos^2(θ)\nWe can rewrite this in Cartesian coordinates using x = rcos(θ) and y = rsin(θ), so r^2 = x^2 + y^2.\nV(x, y) = (1/2)k(x^2 + y^2) + (3/2)kx^2\nV(x, y) = (1/2)kx^2 + (3/2)kx^2 + (1/2)ky^2\nV(x, y) = 2kx^2 + (1/2)ky^2\n\nThis potential represents two independent harmonic oscillators in the x and y directions.\nThe potential in the x-direction is V_x = 2kx^2, and the potential in the y-direction is V_y = (1/2)ky^2.\n\nThe angular frequency for a harmonic oscillator is given by ω = sqrt(k/m).\nFor the x-direction, ω_x = sqrt(2k/m) = sqrt(2) * sqrt(k/m).\nFor the y-direction, ω_y = sqrt(k/(2m)) 

Processing:  14%|█▍        | 21/150 [00:00<00:01, 64.84it/s]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nLet the speed of light in the glass relative to the glass be $v_g$.\nThe index of refraction of the glass is $n$.\nThe speed of light in vacuum is $c=1$.\nSo, $v_g = \\frac{c}{n} = \\frac{1}{n}$.\n\nThe glass is moving with velocity $v$ in the same direction as the light beam.\nLet $v_{obs}$ be the speed of light in the glass relative to the observer in the laboratory.\nAccording to the velocity addition formula,\n$$v_{obs} = \\frac{v_g + v}{1 + \\frac{v_g v}{c^2}}$$\nSince $c=1$,\n$$v_{obs} = \\frac{v_g + v}{1 + v_g v}$$\nSubstituting $v_g = \\frac{1}{n}$,\n$$v_{obs} = \\frac{\\frac{1}{n} + v}{1 + \\frac{1}{n}v} = \\frac{\\frac{1+nv}{n}}{\\frac{n+v}{n}} = \

Processing:  28%|██▊       | 42/150 [00:00<00:01, 80.18it/s]

...

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nHere's how to determine the most strongly reflected wavelength:\n\n1. **Identify the condition for *destructive* interference in the *transmitted* wave:** We are looking for the wavelength that is *least* transmitted, meaning it experiences destructive interference in the transmitted wave, leading to maximal reflection.  Since there is a phase shift of pi (half a wavelength) at the air-oil interface due to the higher refractive index of oil, the condition for *destructive* interference in the *transmitted* wave is given by: 2 * n * d * cos(θ) = m * λ, where n is the refractive index of the film, d is the thickness of the film, θ is the angle of incidence wit

Processing:  99%|█████████▉| 149/150 [08:24<00:06,  6.96s/it]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nHere's how to determine the most strongly reflected wavelength:\n\n1. **Identify the conditions for *constructive* interference in the *reflected* wave:** We are looking for the wavelength that is *most* strongly reflected.  This occurs when the reflected waves interfere constructively.  We need to consider the phase shifts at both interfaces.  At the air-oil interface, there is a phase shift of pi (half a wavelength) because the refractive index of oil is greater than that of air.  At the oil-water interface, there is *no* phase shift since the refractive index of water (1.33) is less than that of oil (1.5).\n\n2. **Condition for constructive interference:*

Processing: 100%|██████████| 150/150 [08:46<00:00,  3.51s/it]

Completed in 526.5 seconds
